- Ivan Živić 993
- Zadatak br. 1

In [1]:
import nltk
import timeit

Rečenice koje će se koristiti za testiranje parsera

In [2]:
test_sent_1 = 'John is eating a donut'
test_sent_2 = 'John likes donuts'
test_sent_3 = ' John is walking at the park'
test_sent_4 = 'He is afraid of monkeys'
test_sent_5 = ' John prefers a morning flight'
test_sent_6 = 'John is leaving from Belgrade'
test_sent_7 = 'John practices volleyball on Thursday'
test_sent_8 = 'I leave Belgrade in the morning'

### <center>Recursive descent parser</center>

**Recursive descent** parser vrši parsiranje metodom "grube sile", odozgo nadole (eng. **top-down**) krećući se od startnog simbola gramatike ka terminalnim čvorovima. U toku generisanja stabla, algoritam "isprobava" sve moguće kombinacije (otud i naziv **recursive**). Upravo je iz ovog razloga algoritam sporiji, ali na izlazu daje sva moguća sintaksna stabla za jedan ulazni niz terminalnih simbola (rečenicu). 

Gramatika navedena ispod definisana je u fajlu pod nazivom **grammar_file_1.cfg**

- S -> N PRE
- PRE -> V N | VP PP | VP NP | V NP | V N PP
- N -> PRP | "John" | "donuts" | "volleyball" | "Thursday" | "Belgrade" | "park" | "monkeys" | "donut" | "flight" | "morning"
- NP -> Det N | Det ADJ N
- V -> "likes" | "practices" | "leaving" | "walking" | "afraid" | "eating" | "prefers" | "leave"
- VP -> Aux V
- Aux -> "is"
- PP -> IN N | IN NP
- IN -> "on" | "from" | "at" | "of" | "in"
- Det -> "the" | "a"
- PRP -> "He" | "I"
- ADJ -> "morning"

#### <center>Osvrt na kreiranu gramatiku</center>

- **S** je startni simbol gramatike. Pravilo **S -> N PRE** ukazuje na to da se rečenica sastoji od imenice **N** koja igra ulogu subjekta u rečenici i predikta **PRE**.
- **PRE** je predikat u rečenici i u svim navedenim primerima dolazi iza subjekta. Dalje, predikat je moguće rastaviti na glagol odnosno imenicu (pravilo **PRE -> V N**). U ovom slučaju glagol (**V**) igra ulogu samog predikta, odnosno glavnog "akcionog" glagola dok **N** koje dolazi iza **V** igra ulogu objekta. Pravilo **PRE -> VP PP** uvedeno je zbog potrebe da predikat na početku sadrži glagolsku frazu (**VP**), a u nastavku imensku frazu (ili samo imenicu) kojoj prethodi predlog (npr. **VP** je **is leaving** a **PP** je **from Belgrade**). Pravilo **PRE -> VP NP** uvedeno je zbog potrebe da se u okviru predikta prvo nađe glagolska a zatim i imenska fraza (npr. **VP** je **is eating**, a **NP** je **a donut**). Pravilo **PRE -> V NP** uvedeno je zbog rečenica kod kojih predikat počinje glagolom, a zatim se nastavlja imenskom frazom (npr. **V** je **prefers**, a **NP** je **a morning flight**). Pravilo **PRE -> V N PP** uvedeno je zbog rečenica čiji predikat najpre počinje glagolom, zatim sledi imenica, a na kraju ide predlog uz imenicu ili imensku frazu (npr. **V** je **practices**, **N** je **volleyball**, a **PP** je **on Thursday**).
- Neterminal **V** igra ulogu "glavnog" glagola (eng. **verb**).
- Neterminal **N** igra ulonu imenice (eng. **noun**).
- Neterminal **Aux** (eng. **auxiliary**) igra ulogu pomoćnog glagola (dodaje se ispred glavnog glagola da bi formirali glagolsku frazu).
- **VP** predstavlja glagolsku frazu koja se sastoji od pomoćnog glagola (predstavljenog neterminalom **Aux**) i glavnog, "akcionog" glagola (**VP -> Aux V**).
- **NP** je imenska fraza. Imenska fraza se sastoji od člana, određenog ili neodređenog (članovi su predstavljeni neterminalom **Det**), atributa (**ADJ**, koji nije obavezan) i imenice. Pravila: **NP -> Det N**, odnosno **Det ADJ N**.
- Neterminal **PP** se preslikava u predlog (predstavljen neterminalom **IN**) i imensku frazu ili samo imenicu. Pravila: **PP -> IN N**, odnosno **IN NP**.
- Neterminalom **IN** predstavljeni su predlozi (**on**, **from**, **at**, **of** i **in**).
- Neterminal **ADJ** (od eng. **adjective**) predstavlja prideve.
- Neterminalom **PRP** predstavljene su zamenice (**He** i **I**).
- Neterminalom **Det** (od eng. **determiners**) predstavljeni su u ovom slučaju samo određeni i neodređeni članovi **a** i **the**.

Pozivom funkcije **load**, koja je definisana u **nltk.data** modulu, učitava se zadata gramatika. Fajl koji sadrži gramatiku, prosleđuje se kao argument funkciji.

In [3]:
grammar_1 = nltk.data.load('file:grammar_file_1.cfg')

Pri instanciranju **RecursiveDescentParser klase**, konstruktoru iste se prosleđuje objekat koji sadrži učitanu gramatiku.

In [4]:
rd_parser = nltk.RecursiveDescentParser(grammar_1)

Pri parsiranju rečenice **John is eating a donut**, algoritam **Recursive descent** dolazi od startnog simbola gramatike **S** do terminalnih čvorova na sledeći način, primenom sledećih smena u navedenom redosledu:

- S -> N PRE
- S -> N VP NP 
- S -> N VP Det N
- S -> N VP Det donut
- S -> N VP a donut
- S -> N Aux V a donut
- S -> N Aux eating a donut
- S -> N is eating a donut
- S -> John is eating a donut

In [5]:
for tree in rd_parser.parse(test_sent_1.split()):
    tree.pretty_print()

               S                     
  _____________|_____                 
 |                  PRE              
 |         __________|_______         
 |        VP                 NP      
 |     ___|____           ___|____    
 N   Aux       V        Det       N  
 |    |        |         |        |   
John  is     eating      a      donut



Pri parsiranju rečenice **John likes donuts** algoritam **Recursive descent** dolazi od startnog simbola gramatike **S** do terminalnih čvorova na sledeći način, primenom sledećih smena u navedenom redosledu:

- S -> N PRE
- S -> N V N
- S -> N V donuts
- S -> N likes donuts
- S -> John likes donuts


In [6]:
for tree in rd_parser.parse(test_sent_2.split()):
    tree.pretty_print()

       S             
  _____|____          
 |         PRE       
 |      ____|____     
 N     V         N   
 |     |         |    
John likes     donuts



Pri parsiranju rečenice **John is walking at the park** algoritam **Recursive descent** dolazi od startnog simbola gramatike **S** do terminalnih čvorova na sledeći način, primenom sledećih smena u navedenom redosledu:

- S -> N PRE
- S -> N VP PP
- S -> N VP IN NP
- S -> N VP IN Det N
- S -> N VP IN Det park
- S -> N VP IN the park
- S -> N VP at the park
- S -> N Aux V at the park
- S -> N Aux walking at the park
- S -> N is wakling at the park
- S -> John is walking at the park


In [7]:
for tree in rd_parser.parse(test_sent_3.split()):
    tree.pretty_print()

                S                    
  ______________|_____                
 |                   PRE             
 |         ___________|___            
 |        |               PP         
 |        |            ___|___        
 |        VP          |       NP     
 |     ___|_____      |    ___|___    
 N   Aux        V     IN Det      N  
 |    |         |     |   |       |   
John  is     walking  at the     park



Pri parsiranju rečenice **He is afraid of monkeys** algoritam **Recursive descent** dolazi od startnog simbola gramatike **S** do terminalnih čvorova na sledeći način, primenom sledećih smena u navedenom redosledu:

- S -> N PRE
- S -> N VP PP
- S -> N VP IN N
- S -> N VP IN monkeys
- S -> N VP of monkeys
- S -> N Aux V of monkeys
- S -> N Aux afraid of monkeys
- S -> N is afraid of monkeys
- S -> PRP is afraid of monkeys
- S -> He is afraid of monkeys

In [8]:
for tree in rd_parser.parse(test_sent_4.split()):
    tree.pretty_print()

              S                       
  ____________|_____                   
 |                 PRE                
 |        __________|_______           
 N       VP                 PP        
 |    ___|____           ___|_____     
PRP Aux       V         IN        N   
 |   |        |         |         |    
 He  is     afraid      of     monkeys



Pri parsiranju rečenice **John prefers a morning flight** algoritam **Recursive descent** dolazi od startnog simbola gramatike **S** do terminalnih čvorova na sledeći način, primenom sledećih smena u navedenom redosledu:

- S -> N PRE
- S -> N V NP
- S -> N V Det ADJ N
- S -> N V Det ADJ flight
- S -> N V Det morning flight
- S -> N V a morning flight
- S -> N prefers a morning flight
- S -> John prefers a morning flight

In [9]:
for tree in rd_parser.parse(test_sent_5.split()):
    tree.pretty_print()

              S                    
  ____________|___                  
 |               PRE               
 |       _________|_____            
 |      |               NP         
 |      |      _________|______     
 N      V    Det       ADJ     N   
 |      |     |         |      |    
John prefers  a      morning flight



Pri parsiranju rečenice **John is leaving from Blegrade** algoritam **Recursive descent** dolazi od startnog simbola gramatike **S** do terminalnih čvorova na sledeći način, primenom sledećih smena u navedenom redosledu:

- S -> N PRE
- S -> N VP PP
- S -> N VP IN N
- S -> N VP IN Belgrade
- S -> N VP from Belgrade
- S -> N Aux V from Belgrade
- S -> N Aux leaving from Belgrade
- S -> N is leaving from Belgrade
- S -> John is leaving from Belgrade


In [10]:
for tree in rd_parser.parse(test_sent_6.split()):
    tree.pretty_print()

                S                         
  ______________|_____                     
 |                   PRE                  
 |         ___________|________            
 |        VP                   PP         
 |     ___|_____           ____|_____      
 N   Aux        V         IN         N    
 |    |         |         |          |     
John  is     leaving     from     Belgrade



Pri parsiranju rečenice **John practices a volleyball on Thursday** algoritam **Recursive descent** dolazi od startnog simbola gramatike **S** do terminalnih čvorova na sledeći način, primenom sledećih smena u navedenom redosledu:

- S -> N PRE
- S -> N V N PP
- S -> N V N IN N
- S -> N V N IN Thursday
- S -> N V N on Thursday
- S -> N V volleyball on Thursday
- S -> N practices volleyball on Thursday
- S -> John practices volleyball on Thursday

In [11]:
for tree in rd_parser.parse(test_sent_7.split()):
    tree.pretty_print()

         S                                
  _______|_________                        
 |                PRE                     
 |        _________|___________            
 |       |         |           PP         
 |       |         |        ___|_____      
 N       V         N       IN        N    
 |       |         |       |         |     
John practices volleyball  on     Thursday



Pri parsiranju rečenice **I leave Belgrade in the morning** algoritam **Recursive descent** dolazi od startnog simbola gramatike **S** do terminalnih čvorova na sledeći način, primenom sledećih smena u navedenom redosledu:

- S -> N PRE
- S -> N V N PP
- S -> N V N IN NP
- S -> N V N IN Det N
- S -> N V N IN Det morning
- S -> N V N IN the morning
- S -> N V N in the morning
- S -> N V Belgrade in the morning
- S -> N leave Belgrade in the morning
- S -> PRP leave Belgrade in the morning
- S -> I leave Belgrade in the morning

In [12]:
for tree in rd_parser.parse(test_sent_8.split()):
    tree.pretty_print()

             S                        
  ___________|______                   
 |                 PRE                
 |     _____________|___               
 |    |      |          PP            
 |    |      |       ___|___           
 N    |      |      |       NP        
 |    |      |      |    ___|_____     
PRP   V      N      IN Det        N   
 |    |      |      |   |         |    
 I  leave Belgrade  in the     morning



### <center>Shift reduce parser</center>

- Ukoliko se primeni gramatika koja je definisana za **Recursive descent** parser, i isti primeni na 7. odnosno 8. rečenici, nećemo dobiti nijedan izlaz. 
- Ukoliko za pomenute rečenice iskortistimo gramatiku koja je navedena u ćeliji ispod i koja je proširena pravilom **S -> S PP**, dobićemo izlaz.
- Sa druge strane, gramatiku definisanu u ćeliji ispod ne možemo upotrebiti za **Recursive descent** algoritam, upravo zbog dodatog pravila koje je rekurzivne prirode, tako da će algoritam prilikom pokušaja da izgeneriše stablo ući u beskonačnu petlju.

Gramatika navedena ispod definisana je u fajlu pod nazivom **grammar_file_2.cfg**

- S -> N PRE | S PP
- PRE -> V N | VP PP | VP NP | V NP | V N PP
- N -> PRP | "John" | "donuts" | "volleyball" | "Thursday" | "Belgrade" | "park" | "monkeys" | "donut" | "flight" | "morning"
- NP -> Det N | Det ADJ N
- V -> "likes" | "practices" | "leaving" | "walking" | "afraid" | "eating" | "prefers" | "leave"
- VP -> Aux V
- Aux -> "is"
- PP -> IN N | IN NP
- IN -> "on" | "from" | "at" | "of" | "in"
- Det -> "the" | "a"
- PRP -> "He" | "I"
- ADJ -> "morning"


In [13]:
grammar_2 = nltk.data.load('file:grammar_file_2.cfg')

Ukoliko definišemo gramatiku tako da sadrži pravila **N -> "morning"** i **ADJ -> "morning"** (za čim svakako postoji potreba, budući da se reč **morning** u rečenici sa rednim brojem 5 javlja kao pridev, a u rečenici sa rednim brojem 8 kao imenica), u ćeliji ispod možemo videti da nas **Shift reduce** parser upozorava da neće primeniti pravilo **N -> "morning"**. Međutim posledice ovako definisane gramatike nećemo videti sve dok ne probamo da parsiramo rečenicu sa rednim brojem 5.

In [14]:
sr_parser = nltk.ShiftReduceParser(grammar_2)

Koraci koje preduzima **Shift reduce** algoritam pri parsiranju rečenice **John is eating a donut**. Poziv algoritma na izvršenje obavljen je u ćeliji ispod.

| No. | Stek | Ulazni niz terminala  | Redukcija / šiftovanje |
| --- | :--- | :---: | :--- |
| 1 | **START** | John is eating a donut | SHIFT |
| 2 | **START** John | is eating a donut | N -> John |
| 3 | **START** N | is eating a donut | SHIFT |
| 4 | **START** N is | eating a donut | aux -> is |
| 5 | **START** N Aux | eating a donut | SHIFT |
| 6 | **START** N Aux eating | a donut | V -> eating |
| 7 | **START** N Aux V | a donut | VP -> Aux V |
| 8 | **START** N VP | a donut | SHIFT |
| 9 | **START** N VP a | donut | Det -> a |
| 10 | **START** N VP Det | donut | SHIFT |
| 11 | **START** N VP Det donut | ... | N -> donut |
| 12 | **START** N VP Det N | ... | NP -> Det N |
| 13 | **START** N VP NP | ... | PRE -> VP NP |
| 14 | **START** N PRE | ... | S -> N PRE |
| 15 | **START** S | ... | acc |


In [15]:
for tree in sr_parser.parse(test_sent_1.split()):
    tree.pretty_print()

               S                     
  _____________|_____                 
 |                  PRE              
 |         __________|_______         
 |        VP                 NP      
 |     ___|____           ___|____    
 N   Aux       V        Det       N  
 |    |        |         |        |   
John  is     eating      a      donut



Koraci koje preduzima **Shift reduce** algoritam pri parsiranju rečenice sa rednim brojem **John likes donuts**. Poziv algoritma na izvršenje obavljen je u ćeliji ispod.

| No. | Stek | Ulazni niz terminala  | Redukcija / šiftovanje |
| --- | :--- | :---: | :--- |
| 1 | **START** | John likes donuts | SHIFT |
| 2 | **START** John | likes donuts | N -> John |
| 3 | **START** N | likes donuts | SHIFT |
| 4 | **START** N likes | donuts | V -> likes |
| 5 | **START** N V | donuts | SHIFT |
| 6 | **START** N V donuts | ... | N -> donuts |
| 7 | **START** N V N | ... | PRE -> V N |
| 8 | **START** N PRE | ... | S -> N PRE |

In [16]:
for tree in sr_parser.parse(test_sent_2.split()):
    tree.pretty_print()

       S             
  _____|____          
 |         PRE       
 |      ____|____     
 N     V         N   
 |     |         |    
John likes     donuts



Koraci koje preduzima **Shift reduce** algoritam pri parsiranju rečenice sa rednim brojem **John is walking at the park**. Poziv algoritma na izvršenje obavljen je u ćeliji ispod.


| No. | Stek | Ulazni niz terminala  | Redukcija / šiftovanje |
| --- | :--- | :---: | :--- |
| 1 | **START** | John is walking at the park | SHIFT |
| 2 | **START** John | is walking at the park | N -> John |
| 3 | **START** N | is walking at the park | SHIFT |
| 4 | **START** N is | walking at the park | aux -> is |
| 5 | **START** N Aux | walking at the park | SHIFT |
| 6 | **START** N Aux walking | at the park | V -> walking |
| 7 | **START** N Aux V | at the park | VP -> Aux V |
| 8 | **START** N VP | at the park | SHIFT |
| 9 | **START** N VP at | the park | IN -> at |
| 10 | **START** N VP IN | the park | SHIFT |
| 11 | **START** N VP IN the | park | Det -> the |
| 12 | **START** N VP IN Det | park | SHIFT |
| 13 | **START** N VP IN Det park | ... | N -> park |
| 14 | **START** N VP IN Det N | ... | NP -> Det N |
| 15 | **START** N VP IN NP | ... | PP -> IN NP |
| 16 | **START** N VP PP | ... | PRE -> VP PP |
| 17 | **START** N PRE | ... | S -> N PRE |
| 18 | **START** S | ... | acc |

In [17]:
for tree in sr_parser.parse(test_sent_3.split()):
    tree.pretty_print()

                S                    
  ______________|_____                
 |                   PRE             
 |         ___________|___            
 |        |               PP         
 |        |            ___|___        
 |        VP          |       NP     
 |     ___|_____      |    ___|___    
 N   Aux        V     IN Det      N  
 |    |         |     |   |       |   
John  is     walking  at the     park



Koraci koje preduzima **Shift reduce** algoritam pri parsiranju rečenice sa rednim brojem **He is afraid of monkeys**. Poziv algoritma na izvršenje obavljen je u ćeliji ispod.

| No. | Stek | Ulazni niz terminala  | Redukcija / šiftovanje |
| --- | :--- | :---: | :--- |
| 1 | **START** | He is afraid of monkeys | SHIFT |
| 2 | **START** He | is afraid of monkeys | PRP -> He |
| 3 | **START** PRP | is afraid of monkeys | N -> PRP |
| 4 | **START** N | is afraid of monkeys | SHIFT |
| 5 | **START** N is | afraid of monkeys | aux -> is |
| 6 | **START** N Aux | afraid of monkeys | SHIFT |
| 7 | **START** N Aux afraid | of monkeys | V -> afraid |
| 8 | **START** N Aux V | of monkeys | VP -> Aux V |
| 9 | **START** N VP | of monkeys | SHIFT |
| 10 | **START** N VP of | monkeys | IN -> of |
| 12 | **START** N VP IN | monkeys | SHIFT |
| 13 | **START** N VP IN monkeys | ... | N -> monkeys |
| 14 | **START** N VP IN N | ... | PP -> IN N |
| 15 | **START** N VP PP | ... | PRE -> VP PP |
| 16 | **START** N PRE | ... | S -> N PRE |
| 17 | **START** S | ... | acc |

In [18]:
for tree in sr_parser.parse(test_sent_4.split()):
    tree.pretty_print()

              S                       
  ____________|_____                   
 |                 PRE                
 |        __________|_______           
 N       VP                 PP        
 |    ___|____           ___|_____     
PRP Aux       V         IN        N   
 |   |        |         |         |    
 He  is     afraid      of     monkeys



Ukoliko probamo da parsiramo rečenicu **John prefers a morning flight**, pomoću kreiranog **Shift reduce** parsera, na osnovu gramatike učitane iz fajla **grammar_file_2.cfg** u promenljivu **grammar_2**, iako nas je parser upozorio da neće koristiti pravilo **N -> "morning"**, izgleda da on u suštini ne koristi pravilo **ADJ -> "morning"**. Do ovakvog zaključka sam došao budući da parser ne može da navedenu rečenicu, budući da kod prikazan u ćeliji ispod ne daje nijedan izlaz.

In [19]:
for tree in sr_parser.parse(test_sent_5.split()):
    tree.pretty_print()

Međutim, ukoliko preuredimo gramatiku koja je sadržana u promenljivoj **grammar_2**, tako da sada ne sadrži pravilo **N -> "morning"**, kao što je učinjeno u ćeliji ispod, možemo videti da će se parser sada "snaći" sa rečenicom **John prefers a morning flight**.

Gramatika navedena ispod definisana je u fajlu pod nazivom **grammar_file_3.cfg**

- S -> N PRE | S PP
- PRE -> V N | VP PP | VP NP | V NP | V N PP
- N -> PRP | "John" | "donuts" | "volleyball" | "Thursday" | "Belgrade" | "park" | "monkeys" | "donut" | "flight"
- NP -> Det N | Det ADJ N
- V -> "likes" | "practices" | "leaving" | "walking" | "afraid" | "eating" | "prefers" | "leave"
- VP -> Aux V
- Aux -> "is"
- PP -> IN N | IN NP
- IN -> "on" | "from" | "at" | "of" | "in"
- Det -> "the" | "a"
- PRP -> "He" | "I"
- ADJ -> "morning"


Kreiranje novog objekta koji predstavlja gramatiku jezika na osnovu gramatike koja je definisana u fajlu **grammar_file_3.cfg**

In [20]:
grammar_3 = nltk.data.load('file:grammar_file_3.cfg')

Kreiranje novog **Shift reduce** parsera na osnovu novokreiranog objekta gramatike.

In [21]:
sr_parser = nltk.ShiftReduceParser(grammar_3)

Imajući u vidu da je gramatika sada preuređena, **Shift reduce** parser se sada "snalazi" sa rečenicom **John prefers a morning flight** (u ovoj rečenici reč **morning** je pridev **ADJ**). 

Koraci koje preduzima **Shift reduce** algoritam pri parsiranju rečenice **John prefers a morning flight**. Poziv algoritma na izvršenje obavljen je u ćeliji ispod.

| No. | Stek | Ulazni niz terminala  | Redukcija / šiftovanje |
| --- | :--- | :---: | :--- |
| 1 | **START** | John prefers a morning flight | SHIFT |
| 2 | **START** John | prefers a morning flight | SHIFT |
| 3 | **START** N | prefers a morning flight | SHIFT |
| 4 | **START** N prefers | a morning flight | V -> prefers |
| 5 | **START** N V | a morning flight | SHIFT |
| 6 | **START** N V a | morning flight | Det -> a |
| 7 | **START** N V Det | morning flight | SHIFT |
| 8 | **START** N V Det morning | flight | ADJ -> morning |
| 9 | **START** N V Det ADJ | flight | SHIFT |
| 10 | **START** N V Det ADJ flight | ... | N -> flight |
| 11 | **START** N V Det ADJ N | ... | NP -> Det ADJ N |
| 12 | **START** N V NP | ... | PRE -> V NP |
| 13 | **START** N PRE | ... | S -> N PRE |
| 14 | **START** S | ... | acc |

In [22]:
for tree in sr_parser.parse(test_sent_5.split()):
    tree.pretty_print()

              S                    
  ____________|___                  
 |               PRE               
 |       _________|_____            
 |      |               NP         
 |      |      _________|______     
 N      V    Det       ADJ     N   
 |      |     |         |      |    
John prefers  a      morning flight



Koraci koje preduzima **Shift reduce** algoritam pri parsiranju rečenice **John is leaving from Belgrade**. Poziv algoritma na izvršenje obavljen je u ćeliji ispod.

| No. | Stek | Ulazni niz terminala  | Redukcija / šiftovanje |
| --- | :--- | :---: | :--- |
| 1 | **START** | John is leaving from Belgrade | SHIFT |
| 2 | **START** John | is leaving from Belgrade | N -> John |
| 3 | **START** N | is leaving from Belgrade | SHIFT |
| 4 | **START** N is | leaving from Belgrade | aux -> is |
| 5 | **START** N Aux | leaving from Belgrade | SHIFT |
| 6 | **START** N Aux leaving | from Belgrade | V -> leaving |
| 7 | **START** N Aux V | from Belgrade | VP -> Aux V |
| 8 | **START** N VP | from Belgrade | SHIFT |
| 9 | **START** N VP from | Belgrade | IN -> from |
| 10 | **START** N VP IN | Belgrade | SHIFT |
| 11 | **START** N VP IN Belgrade | ... | N -> Belgrade |
| 12 | **START** N VP IN N | ... | PP -> IN N |
| 13 | **START** N VP PP | ... | PRE -> VP PP |
| 14 | **START** N PRE | ... | S -> N PRE |
| 15 | **START** S | ... | acc |


In [23]:
for tree in sr_parser.parse(test_sent_6.split()):
    tree.pretty_print()

                S                         
  ______________|_____                     
 |                   PRE                  
 |         ___________|________            
 |        VP                   PP         
 |     ___|_____           ____|_____      
 N   Aux        V         IN         N    
 |    |         |         |          |     
John  is     leaving     from     Belgrade



Koraci koje preduzima **Shift reduce** algoritam pri parsiranju rečenice **John practices volleyball on Thursday**. Poziv algoritma na izvršenje obavljen je u ćeliji ispod.

Pokušaj da se iskoristi gramatika definisana za **Recursive descent** algoritam (gramatika sadržana u promenljivoj **grammar_1**):

| No. | Stek | Ulazni niz terminala  | Redukcija / šiftovanje |
| --- | :--- | :---: | :--- |
| 1 | **START** | John practices volleyball on Thursday | SHIFT |
| 2 | **START** John | practices volleyball on Thursday | N -> John |
| 3 | **START** N | practices volleyball on Thursday | SHIFT |
| 4 | **START** N practices | volleyball on Thursday | V -> practices |
| 5 | **START** N V | volleyball on Thursday | SHIFT |
| 6 | **START** N V volleyball | on Thursday | N -> volleyball |
| 7 | **START** N V N | on Thursday | PRE -> V N |
| 8 | **START** N PRE | on Thursday | SHIFT |
| 9 | **START** S | on Thursday | SHIFT |
| 10 | **START** S on | Thursday | IN -> on |
| 11 | **START** S IN | Thursday | SHIFT |
| 12 | **START** S IN Thursday | ... | N -> Thursday |
| 13 | **START** S IN N | ... | PP -> IN N |
| 14 | **START** S PP | ... | ... |

Ovde će se algoritam zaustaviti, budući da je ulazni niz terminalnih simbola prazan, a stek i dalje sadrži elemente. Međutim, ukoliko uvedemo pravilo **S -> S PP** koje je "pogubno" za **Recursive descent** algoritam, omogućićemo **Shift reduce** algoritmu da izvrši parsiranje. 

Uvođenjem pravila **S -> S PP**, kao što je to učinjeno u gramatikama sadržanim u promenljivama **grammar_2** i **grammar_3**, prevazilazimo ovaj problem.

| No. | Stek | Ulazni niz terminala  | Redukcija / šiftovanje |
| --- | :--- | :---: | :--- |
| 15 | **START** S PP | ... | S -> S PP |
| 16 | **START** S | ... | acc |


In [24]:
for tree in sr_parser.parse(test_sent_7.split()):
    tree.pretty_print()

                       S                      
          _____________|___________            
         S                         |          
  _______|______                   |           
 |             PRE                 PP         
 |        ______|______         ___|_____      
 N       V             N       IN        N    
 |       |             |       |         |     
John practices     volleyball  on     Thursday



**U rečenici **John prefers a morning flight**, reč **morning** predstavlja pridev, dok u rečenici **I leave Belgrade in the morning** predstavlja imenicu. Ukoliko iskoristimo gramatiku na način kako je navedena u promenljivoj pod nazivom **grammar_2**, **Shift reduce** parser neće uspeti da parsira rečenicu **I leave Belgrade in the morning**, budući da pomenuti parser, za razliku od **Recursive descent** parsera ne dozvoljava da se navedu dva pravila: **ADJ -> "morning"** i **N -> "morning"**. Ukoliko želimo da parser uspe da parsira rečenicu sa rednim brojem 8, moraćemo da eliminišemo pravilo **ADJ -> "morning"** i dodamo pravilo **N -> "morning"**, kao što je prikazano u ćeliji ispod u promenljivoj pod nazivom **grammar_4** u koju je učitana gramatika navedena u fajlu **grammar_file_4.cfg**.

- S -> N PRE | S PP
- PRE -> V N | VP PP | VP NP | V NP | V N PP
- N -> PRP | "John" | "donuts" | "volleyball" | "Thursday" | "Belgrade" | "park" | "monkeys" | "donut" | "flight" | "morning"
- NP -> Det N | Det ADJ N
- V -> "likes" | "practices" | "leaving" | "walking" | "afraid" | "eating" | "prefers" | "leave"
- VP -> Aux V
- Aux -> "is"
- PP -> IN N | IN NP
- IN -> "on" | "from" | "at" | "of" | "in"
- Det -> "the" | "a"
- PRP -> "He" | "I"

In [25]:
grammar_4 = nltk.data.load('file:grammar_file_4.cfg')

In [26]:
sr_parser = nltk.ShiftReduceParser(grammar_4)

Koraci koje preduzima **Shift reduce** algoritam pri parsiranju rečenice **I leave Belgrade in the morning**. Poziv algoritma na izvršenje obavljen je u ćeliji ispod.

| No. | Stek | Ulazni niz terminala  | Redukcija / šiftovanje |
| --- | :--- | :---: | :--- |
| 1 | **START** | I leave Belgrade in the morning | SHIFT |
| 2 | **START** I | leave Belgrade in the morning | PRP -> I |
| 3 | **START** PRP | leave Belgrade in the morning | N -> PRP |
| 4 | **START** N | leave Belgrade in the morning | SHIFT |
| 5 | **START** N leave | Belgrade in the morning | V -> leave |
| 6 | **START** N V | Belgrade in the morning | SHIFT |
| 7 | **START** N V Belgrade | in the morning | N -> Belgrade |
| 8 | **START** N V N | in the morning | PRE -> V N |
| 9 | **START** N PRE | in the morning | S -> N PRE |
| 10 | **START** S | in the morning | SHIFT |
| 11 | **START** S in | the morning | IN -> in |
| 12 | **START** S IN | the morning | SHIFT |
| 13 | **START** S IN the | morning | Det -> the |
| 14 | **START** S IN Det | morning | SHIFT |
| 15 | **START** S IN Det morning | ... | N -> morning |
| 16 | **START** S IN Det N | ... | NP -> Det N |
| 17 | **START** S IN NP | ... | PP -> IN NP |
| 18 | **START** S PP | ... | S -> S PP |
| 19 | **START** S | ... | acc |

In [27]:
for tree in sr_parser.parse(test_sent_8.split()):
    tree.pretty_print()

                 S                        
       __________|__________               
      S                     PP            
  ____|____              ___|___           
 N        PRE           |       NP        
 |     ____|_____       |    ___|_____     
PRP   V          N      IN Det        N   
 |    |          |      |   |         |    
 I  leave     Belgrade  in the     morning



### <center>Merenje vremena za koje algoritmi Recursive descent i Shift reduce vrše parsiranje</center>

- Za potrebe merenja vremena parsiranja koje vrše algoritmi **Recursive descent** i **Shift reduce** algoritmi upotrebljena je biblioteka **timeit**. U ovoj biblioteci definisana je klasa pod nazivom **Timer**, čijem se konstruktoru prosleđuje **closure** definisan u ćeliji ispod. Spoljna funkcija **closure**-a prihvata dva argumenta. Prvi argument je parser objekat, a drugi argument je rečenica koju želimo parsirati. **Closure** kao povratnu vrednost vraća unutrašnju funkciju (nazvanu **inner_fun**) koja pri pozivu može da koristi oba argumenta prosleđena spoljašnjoj funkciji (**timer_fun**).
- U unutrašnjoj funkciji **closure-a** poziva se **parse** metoda prosleđenog parsera, a "promenljiva odbacivanja" u **for** petlji i **pass** u telu iste se koriste da se jednostavno ne štampaju stabla kao izlazi (iako su stabla kreirana), da bi se na izlazu prikazalo samo vreme parsiranja **for _ in parser_obj.parse(test_sent.parse())**.
- Pri merenju vremena parsiranja svake zadate test rečenice, **timeit** metodi objekta **Timer** prosleđuje se kao argument broj koji predstavlja koliko puta treba pozvati unutrašnju funkciju **closure**-a na izvršenje. Ovo se obavlja iz razloga što objektivnijeg merenja parsiranja iste rečenice. Naime, parsiranje iste rečenice potrebno je izvršiti veći broj puta (u konkretnim primerima po 1000 puta) i sva dobijena vremena usrednjiti u cilju dobijanja prosečnog vremena parsiranja iste test rečenice.
- Ispod svake ćelije u kojoj se poziva **timeit** metoda **Timer** objekta na izvršenje, prikazano je prosečno vreme parsiranja svake test rečenice.

In [28]:
def timer_fun(parser_obj, test_sent):
    def inner_fun():
        for _ in parser_obj.parse(test_sent.split()):
            pass
    return inner_fun

Najpre se vrši merenje vremena parsiranja test rečenica **Recursive descent** algoritmom. Prosečno vreme parsiranja prikazano ispod svake ćelije dato je u sekundama.

In [29]:
rd_parser = nltk.RecursiveDescentParser(grammar_1)

In [30]:
t = timeit.Timer(timer_fun(rd_parser, test_sent_1))
t.timeit(1000)

2.1582741829988663

In [31]:
t = timeit.Timer(timer_fun(rd_parser, test_sent_2))
t.timeit(1000)

1.8763854619974154

In [32]:
t = timeit.Timer(timer_fun(rd_parser, test_sent_3))
t.timeit(1000)

2.879163794001215

In [33]:
t = timeit.Timer(timer_fun(rd_parser, test_sent_4))
t.timeit(1000)

2.4716990380002244

In [34]:
t = timeit.Timer(timer_fun(rd_parser, test_sent_5))
t.timeit(1000)

2.289547421998577

In [35]:
t = timeit.Timer(timer_fun(rd_parser, test_sent_6))
t.timeit(1000)

2.3738096079978277

In [36]:
t = timeit.Timer(timer_fun(rd_parser, test_sent_7))
t.timeit(1000)

2.506304458998784

In [37]:
t = timeit.Timer(timer_fun(rd_parser, test_sent_8))
t.timeit(1000)

3.6081442210015666

Dalje se vrši merenje vremena parsiranja zadatih test rečenica **Shift reduce** algoritmom i računanje prosečnog vremena parsiranja za svaku zadatu rečenicu ponaosob.

In [38]:
sr_parser = nltk.ShiftReduceParser(grammar_2)

In [39]:
t = timeit.Timer(timer_fun(sr_parser, test_sent_1))
t.timeit(1000)

0.25146391700036475

In [40]:
t = timeit.Timer(timer_fun(sr_parser, test_sent_2))
t.timeit(1000)

0.10469948000172735

In [41]:
t = timeit.Timer(timer_fun(sr_parser, test_sent_3))
t.timeit(1000)

0.31069795599978534

In [42]:
t = timeit.Timer(timer_fun(sr_parser, test_sent_4))
t.timeit(1000)

0.2737211829989974

In [43]:
sr_parser = nltk.ShiftReduceParser(grammar_3)

In [44]:
t = timeit.Timer(timer_fun(sr_parser, test_sent_5))
t.timeit(1000)

0.2260773939997307

In [45]:
t = timeit.Timer(timer_fun(sr_parser, test_sent_6))
t.timeit(1000)

0.23317354299797444

In [46]:
t = timeit.Timer(timer_fun(sr_parser, test_sent_7))
t.timeit(1000)

0.19641396199949668

In [47]:
sr_parser = nltk.ShiftReduceParser(grammar_4)

In [48]:
t = timeit.Timer(timer_fun(sr_parser, test_sent_8))
t.timeit(1000)

0.31194595499982825

Kao što je i za očekivati, parsiranje rečenice **Recursive descent** algoritmom iziskuje više vremena nego parsiranje **Shift reduce** algoritmom. Ovo je posledica "rekurzivne" prirode **Recursive descent** algoritma, koji kreće od startnog simbola gramatike i proba pomoću svih zadatih smena da dođe do niza terminalnih simbola.
Ukoliko uporedimo prosečna vremena parsiranja odgovarajućih rečenica, možemo videti da je prosečno vreme parsiranja rečenice **Recursive descent** algoritmom uvek veće od prosečnog vremena parsiranja rečenice **Shift reduce** algoritmom.